<a href="https://colab.research.google.com/github/kavithadaiya/CODSOFT/blob/main/data_deduplication.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import time
import os

from sklearn.metrics import f1_score, accuracy_score
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv("Input.csv")
data['is_duplicate'].value_counts().plot(kind='bar')

FileNotFoundError: [Errno 2] No such file or directory: 'Input.csv'

In [ ]:
data

In [ ]:
data.head()

In [ ]:
#### The dob is converted to standard datetime format.

data.dob = pd.to_datetime(data.dob)

In [ ]:
data.head()

In [ ]:
data.dob.head(10)

In [ ]:
data['name'] = data.fn + ' ' + data.ln

In [ ]:
import hashlib
import base64
data = data.assign(concat  = data.dob.astype(str) + data.gn + data.fn + data.ln)
data['hash']=data['concat'].astype(str).str.encode('UTF-8').apply(lambda x: base64.b64encode(hashlib.md5(x).digest()))
data
#data2=data
data.head()

In [ ]:
#### A list of unique dates of birth and unique genders is obtained.

unique_dob = data.dob.unique()
unique_sex = data.gn.unique()
unique_hash = data.hash.unique()

In [ ]:
pip install Distance

In [ ]:
import distance

In [ ]:
### Deduplication model function. This function learns by evaluating the Macro F1-score of classification on a passed range of values for maximum levenshtein score to classify an entry as duplicate.
start_h= time.time()

def deduplication_model(data, scoring_range = 10, step = 2):
    data['indices'] = list(range(len(data)))
    accuracy = []
    index = []
    final_step = 0
    for value in range(scoring_range):
        for i in unique_dob:
            for j in unique_sex:
                sample = data[(data.dob == i)][(data.gn == j)].reset_index(drop = True)
                for a in range(len(sample)):
                    comparison = sample[(sample.indices != sample.indices[a])].reset_index(drop = True)
                    scores = [distance.levenshtein(sample.name[a], comparison.name[x]) for x in range(len(comparison))]
                    compare = [comparison.indices[x] for x in range(len(comparison))]
                    try:
                        if sample.indices[a]>compare[scores.index(min(scores))]:
                            score = np.min(scores)
                            if score<=value:
                                index.append(sample.indices[a])
                    except ValueError:
                        pass
        prediction = []
        for k in range(len(data)):
            if data.indices[k] in index:
                prediction.append(1)
            else:
                prediction.append(0)

        data['prediction'] = prediction
        print('F1-score after ',value, 'iterations : ', f1_score(data.is_duplicate, data.prediction, average = 'macro'))
        accuracy.append(f1_score(data.is_duplicate, data.prediction, average = 'macro'))
        if len(accuracy)>1 and accuracy[-1] <= accuracy[-2]:
            final_step+=1
        if final_step>=step:
            value = value-final_step
            break

    index = []
    for i in unique_dob:
        for j in unique_sex:
            sample = data[(data.dob == i)][(data.gn == j)].reset_index(drop = True)
            for a in range(len(sample)):
                comparison = sample[(sample.indices != sample.indices[a])].reset_index(drop = True)
                scores = [distance.levenshtein(sample.name[a], comparison.name[x]) for x in range(len(comparison))]
                compare = [comparison.indices[x] for x in range(len(comparison))]
                try:
                    if sample.indices[a]>compare[scores.index(min(scores))]:
                        score = np.min(scores)
                        if score<=value:
                            index.append(sample.indices[a])
                except ValueError:
                    pass
    prediction = []
    for k in range(len(data)):
        if data.indices[k] in index:
            prediction.append(1)
        else:
            prediction.append(0)
    return prediction, value

In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(data, test_size = 0.05, stratify = data.is_duplicate, random_state = 0)
train = train.reset_index(drop = True)
test = test.reset_index(drop = True)
performance, levenshtein_value_optimum = deduplication_model(train, scoring_range = 10, step = 3)

In [ ]:
def deduplication_prediction(data, optimum_value):
    data['indices'] = list(range(len(data)))
    index = []
    for i in unique_dob:
        for j in unique_sex:
            sample = data[(data.dob == i)][(data.gn == j)].reset_index(drop = True)
            for a in range(len(sample)):
                comparison = sample[(sample.indices != sample.indices[a])].reset_index(drop = True)
                scores = [distance.levenshtein(sample.name[a], comparison.name[x]) for x in range(len(comparison))]
                compare = [comparison.indices[x] for x in range(len(comparison))]
                try:
                    if sample.indices[a]>compare[scores.index(min(scores))]:
                        score = np.min(scores)
                        if score<=optimum_value:
                            index.append(sample.indices[a])
                except ValueError:
                    pass
    prediction = []
    for k in range(len(data)):
        if data.indices[k] in index:
            prediction.append(1)
        else:
            prediction.append(0)
    return prediction

In [ ]:
predictions = deduplication_prediction(test, levenshtein_value_optimum)


In [ ]:
print('F1-score on test set:',accuracy_score(test.is_duplicate, predictions))


In [ ]:
train['prediction'] = performance
test['prediction'] = predictions
dataset = pd.concat([train, test], axis = 0)
dataset = dataset[(dataset.prediction != 1)].reset_index(drop = True).drop(labels = ['name', 'is_duplicate', 'prediction', 'indices'], axis = 1)

In [ ]:
dataset.to_csv('output1.csv', index = False)
end_h = time.time()
tt2 = end_h - start_h
print('Time taken: ')
tt2

In [ ]:
data1 = pd.read_csv('output1.csv')
data1.head()

In [ ]:
data1